# Algoritmo de Machine Learging: Mineria de Texto

In [ ]:
# Paso 1: Importar Librerias
import os
import tarfile
import requests
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
# Paso 2: Descargar y descomprimir el dataset IMDB
# URL del dataset IMDB
url = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
dataset_path = 'aclImdb_v1.tar.gz'
extract_path = 'aclImdb'

# Descargar el archivo
response = requests.get(url, stream=True)
with open(dataset_path, 'wb') as file:
    file.write(response.raw.read())

# Descomprimir el archivo
with tarfile.open(dataset_path, 'r:gz') as tar:
    tar.extractall(path=extract_path)

In [ ]:
# Paso 3: Cargar y Prepocesar el Dataset IMDB
# Cargar los datos de entrenamiento y prueba desde las carpetas
data_folder_train = os.path.join(extract_path, 'aclImdb', 'train')
data_folder_test = os.path.join(extract_path, 'aclImdb', 'test')

train_data = load_files(data_folder_train, categories=['pos', 'neg'])
test_data = load_files(data_folder_test, categories=['pos', 'neg'])

# Extraer las características y etiquetas
X_train, y_train = train_data.data, train_data.target
X_test, y_test = test_data.data, test_data.target

In [ ]:
# Paso 4: Construir el Modelo Naive Bayes con un Pipeline
# Construir un pipeline que incluye la vectorización, transformación TF-IDF y el clasificador Naive Bayes
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words=stopwords.words('english'))),  # Convertir texto en una matriz de términos
    ('tfidf', TfidfTransformer()),  # Convertir recuentos en frecuencias tf-idf
    ('clf', MultinomialNB()),  # Clasificador Naive Bayes
])

# Entrenar el modelo con los datos de entrenamiento
text_clf.fit(X_train, y_train)

In [ ]:
# Paso 5: Evaluar el Modelo
# Hacer predicciones en el conjunto de prueba
y_pred = text_clf.predict(X_test)

# Evaluar la precisión del modelo
accuracy = metrics.accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy:.2f}')

# Mostrar un reporte de clasificación más detallado
print(metrics.classification_report(y_test, y_pred, target_names=train_data.target_names))

# Mostrar la matriz de confusión
print(metrics.confusion_matrix(y_test, y_pred))

In [ ]:
# Paso 6: Hacer predicciones con el Modelo
# Ejemplo de cómo hacer predicciones con el modelo entrenado
sample_reviews = [
    "This movie was fantastic!",
    "Terrible movie."
]

predicted = text_clf.predict(sample_reviews)

for review, category in zip(sample_reviews, predicted):
    print(f'Review: {review}\nPredicted Sentiment: {"positive" if category == 1 else "negative"}\n')
